In [1]:
import pandas as pd
import torch

In [2]:
predictions = pd.read_csv("dataset/validations/predictions.csv")
predictions = predictions.rename(
    columns={"x_min": "xmin", "y_min": "ymin", "x_max": "xmax", "y_max": "ymax"}
)
predictions.sample(5)

,Image_ID,confidence,class_id,class,xmin,ymin,xmax,ymax
266,ID_TSkjv2.jpg,0.281757,1,cssvd,499.239227,452.481506,1041.353760,802.661560
387,ID_vDdYaf.jpg,0.070901,1,cssvd,776.963257,800.122070,916.391479,1062.179565
452,ID_esabf4.jpg,0.069914,1,cssvd,49.584312,176.234238,700.086792,368.191528
112,ID_v1PbJC.jpg,0.112268,0,anthracnose,433.405609,1888.725098,1036.146729,2460.729980
48,ID_jTWHzU.jpeg,0.104530,0,anthracnose,1479.855957,41.564972,2448.331787,1528.460693


In [3]:
converted_labels = predictions["Image_ID"].unique()

In [4]:
def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels}

In [5]:
preds = get_preds_data(predictions, thr=None)
len(preds)

56

In [6]:
validations = pd.read_csv("dataset/Val_df.csv")
validations.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id,ImagePath
88,ID_hRfHto.jpeg,healthy,1.0,0.0,0.0,3888.0,2384.0,2,dataset/images/train/ID_hRfHto.jpeg
31,ID_VBvSm2.jpeg,cssvd,1.0,587.0,365.0,951.0,559.0,1,dataset/images/train/ID_VBvSm2.jpeg
2,ID_yZDVIT.jpg,healthy,1.0,1.0,96.0,131.0,258.0,2,dataset/images/train/ID_yZDVIT.jpg
78,ID_RNAyxb.jpg,healthy,1.0,337.0,70.0,616.0,706.0,2,dataset/images/train/ID_RNAyxb.jpg
57,ID_K6JkTA.jpg,cssvd,1.0,665.0,1265.0,1554.0,1780.0,1,dataset/images/train/ID_K6JkTA.jpg


In [7]:
vals = get_preds_data(validations, thr=None)
len(vals)

56

In [8]:
vals["ID_B9K2SI.jpg"]

{'boxes': tensor([[   0.,   24., 1251.,  574.],
         [ 114.,  544.,  351.,  745.]]),
 'scores': tensor([1., 1.]),
 'labels': tensor([1, 1], dtype=torch.int32)}

In [9]:
import torch

def calculate_iou_tensor(box1, box2):
	"""
	box1: [4], box2: [4]
	Format: [xmin, ymin, xmax, ymax]
	"""
	xA = torch.max(box1[0], box2[0])
	yA = torch.max(box1[1], box2[1])
	xB = torch.min(box1[2], box2[2])
	yB = torch.min(box1[3], box2[3])

	inter_area = torch.clamp(xB - xA, min=0) * torch.clamp(yB - yA, min=0)
	box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
	box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
	union_area = box1_area + box2_area - inter_area
	return inter_area / union_area if union_area > 0 else torch.tensor(0.0)

def evaluate_detection(predictions, ground_truths, iou_threshold=0.5, conf_threshold=0.0):
	"""
	predictions: list of dicts (len = batch size), each dict with 'boxes', 'scores', 'labels'
	ground_truths: list of dicts with 'boxes', 'labels'
	"""
	TP = 0
	FP = 0
	FN = 0

	for preds, gts in zip(predictions, ground_truths):
		pred_boxes = preds['boxes']
		pred_labels = preds['labels']
		pred_scores = preds['scores'] if preds['scores'] is not None else torch.ones(len(pred_boxes))

		gt_boxes = gts['boxes']
		gt_labels = gts['labels']
		matched_gt = set()

		for i in range(len(pred_boxes)):
			if pred_scores[i] < conf_threshold:
				continue
			pred_box = pred_boxes[i]
			pred_label = pred_labels[i]
			match_found = False

			for j in range(len(gt_boxes)):
				if j in matched_gt:
					continue
				if pred_label != gt_labels[j]:
					continue
				iou = calculate_iou_tensor(pred_box, gt_boxes[j])
				if iou >= iou_threshold:
					TP += 1
					matched_gt.add(j)
					match_found = True
					break
			if not match_found:
				FP += 1

		FN += len(gt_boxes) - len(matched_gt)

	precision = TP / (TP + FP) if (TP + FP) else 0.0
	recall = TP / (TP + FN) if (TP + FN) else 0.0
	f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0
	accuracy = TP / (TP + FP + FN) if (TP + FP + FN) else 0.0

	return {
		'TP': TP,
		'FP': FP,
		'FN': FN,
		'Precision': precision,
		'Recall': recall,
		'F1 Score': f1_score,
		'Accuracy': accuracy
	}

In [10]:
import numpy as np

for i in np.linspace(0.0, 0.95, 15):
	scores = evaluate_detection(
		preds.values(),
		vals.values(),
		iou_threshold=0.5,
		conf_threshold=i
	)
	print("Evaluation metric at:", i, " score :", scores)

Evaluation metric at: 0.0  score : {'TP': 82, 'FP': 539, 'FN': 13, 'Precision': 0.1320450885668277, 'Recall': 0.8631578947368421, 'F1 Score': 0.22905027932960895, 'Accuracy': 0.12933753943217666}
Evaluation metric at: 0.06785714285714285  score : {'TP': 82, 'FP': 370, 'FN': 13, 'Precision': 0.18141592920353983, 'Recall': 0.8631578947368421, 'F1 Score': 0.2998171846435101, 'Accuracy': 0.17634408602150536}
Evaluation metric at: 0.1357142857142857  score : {'TP': 82, 'FP': 126, 'FN': 13, 'Precision': 0.3942307692307692, 'Recall': 0.8631578947368421, 'F1 Score': 0.5412541254125413, 'Accuracy': 0.37104072398190047}
Evaluation metric at: 0.20357142857142857  score : {'TP': 77, 'FP': 69, 'FN': 18, 'Precision': 0.5273972602739726, 'Recall': 0.8105263157894737, 'F1 Score': 0.6390041493775933, 'Accuracy': 0.4695121951219512}
Evaluation metric at: 0.2714285714285714  score : {'TP': 75, 'FP': 35, 'FN': 20, 'Precision': 0.6818181818181818, 'Recall': 0.7894736842105263, 'F1 Score': 0.731707317073170

In [11]:
import torch
from torchmetrics.detection import MeanAveragePrecision

def compute_map(preds, targets, iou_thresholds):
	"""
	Compute mAP at different IoU thresholds using torchmetrics.
	
	Args:
		preds: List of dicts with 'boxes', 'scores', 'labels' for predictions
		targets: List of dicts with 'boxes', 'labels' for ground truth
		iou_thresholds: List of IoU thresholds to evaluate
	
	Returns:
		Dict containing mAP results for each IoU threshold
	"""
	# Initialize the metric
	metric = MeanAveragePrecision(iou_thresholds=iou_thresholds)
	
	# Update metric with predictions and targets
	metric.update(preds, targets)
	
	# Compute the results
	result = metric.compute()
	
	return result

In [12]:
thrs = np.linspace(0.0, 0.95, 15)
# Example usage
# if __name__ == "__main__":
# Example predictions and targets
targets = list(vals.values())

iou_thresholds = [0.5]
for i in thrs:
	preds = list(get_preds_data(predictions, i).values())

	# Compute mAP
	results = compute_map(preds, targets, iou_thresholds)

	# Print results
	print("mAP Results:", i, " - ", results)

mAP Results: 0.0  -  {'map': tensor(0.7485), 'map_50': tensor(0.7485), 'map_75': tensor(-1.), 'map_small': tensor(-1.), 'map_medium': tensor(-1.), 'map_large': tensor(0.7531), 'mar_1': tensor(0.5214), 'mar_10': tensor(0.8644), 'mar_100': tensor(0.8644), 'mar_small': tensor(-1.), 'mar_medium': tensor(-1.), 'mar_large': tensor(0.8644), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2], dtype=torch.int32)}
mAP Results: 0.06785714285714285  -  {'map': tensor(0.7485), 'map_50': tensor(0.7485), 'map_75': tensor(-1.), 'map_small': tensor(-1.), 'map_medium': tensor(-1.), 'map_large': tensor(0.7531), 'mar_1': tensor(0.5214), 'mar_10': tensor(0.8644), 'mar_100': tensor(0.8644), 'mar_small': tensor(-1.), 'mar_medium': tensor(-1.), 'mar_large': tensor(0.8644), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2], dtype=torch.int32)}
mAP Results: 0.1357142857142857  -  {'map': tensor(0.7485), 'map_50': tensor(0.7485), 